In [1]:
import features as ft
t_data = ft.gen_transfered_dist_feature()[:50000]

S:\anaconda\envs\tf\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
S:\anaconda\envs\tf\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


read the data for transfered_dist_features_train
data for transfered_dist_features_train read


In [9]:
import pandas as pd
import numpy as np
from time import time

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
import sklearn

from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
%matplotlib inline
import matplotlib.pyplot as pl
import matplotlib.patches as mpatches
import numpy as np
from IPython import get_ipython

rand_number = 42



In [33]:
def one_hot(y):
    oc = OneHotEncoder(categories='auto')
    y_r = y.reshape(-1,1)
    oc.fit(y_r)
    r = oc.transform(y_r).toarray()
    return r


def get_log_loss(y_true, y_pred):
    y_p = one_hot(y_pred)
    return log_loss(y_true, y_p)


def t_model(gbm):
    y_trainp = gbm.predict(X_train)
    y_testp  = gbm.predict(X_test)
    print(get_log_loss(y_train, y_trainp), get_log_loss(y_test, y_testp))


def tt_model(gbm):
    gbm = gbm.fit(X_train, y_train)
    t_model(gbm)

out_features = ['question1', 'question2', 'is_duplicate']
features = [x for x in t_data.columns if x not in out_features]

t_data = t_data[t_data!=np.inf].dropna()
feature_data = t_data.drop_duplicates(features, keep='last')
input_data = feature_data[features]
input_data = input_data.astype(np.float64)
result = feature_data[['is_duplicate']]
result = result.values.ravel()

def time_cnt(f, tag="func"):
    t_start = time()
    ret = f()
    t_end = time()
    t_used = t_end - t_start
    print("function '%s' use: %f s" % (tag, t_used))
    
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_data, result, test_size = 0.2, random_state = 0,
                                                    stratify = result)

In [30]:
# sorted(sklearn.metrics.SCORERS.keys())
X_train.shape, X_test.shape

((39809, 164), (9953, 164))

In [31]:
# base model
model = LGBMClassifier(num_thread=8)
test_model(model)

5.587422944138884 7.717697046345962


In [32]:
from sklearn.model_selection import GridSearchCV
params={
    'max_depth': [15, 20, 25, 30, 35],
    'num_leaves': [20, 35, 50]
}

# params = {
#               'max_depth': [15, 20, 25, 30, 35],
#               'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
#               'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
#               'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
#               'bagging_freq': [2, 4, 5, 6, 8],
#               'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
#               'lambda_l2': [0, 10, 15, 35, 40],
#               'cat_smooth': [1, 10, 15, 20, 35]
# }
gbm = LGBMClassifier(num_thread=8)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  45 out of  45 | elapsed:  2.4min finished


{'max_depth': 20, 'num_leaves': 50} -0.4357211897392911


In [34]:
best = LGBMClassifier(num_thread=8, n_jobs=8, max_depth=20, num_leaves=50)
tt_model(best)

4.390985639796101 7.502545420053944


In [38]:
params={
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'bagging_freq': [20, 35, 50, 70]
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done  60 out of  60 | elapsed:  1.6min finished


{'bagging_freq': 20, 'learning_rate': 0.1} -0.4357211897392911


In [43]:
params={
    'bagging_freq': [15, 10, 5, 2]
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50, learning_rate=0.1)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:   18.8s finished


{'bagging_freq': 15} -0.4357211897392911
4.390985639796101 7.502545420053944


In [44]:
params={
    'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
    'lambda_l2': [0, 10, 15, 35, 40],
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50, learning_rate=0.1)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  1.3min
[Parallel(n_jobs=3)]: Done  75 out of  75 | elapsed:  2.1min finished


{'lambda_l1': 0.4, 'lambda_l2': 10} -0.43348030064365295
4.724148747024257 7.426201294595485


In [46]:
params={
    'lambda_l1': [0.33, 0.3, 0.38],
    'lambda_l2': [6, 7, 8, 9],
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50, learning_rate=0.1)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  36 out of  36 | elapsed:  1.1min finished


{'lambda_l1': 0.3, 'lambda_l2': 8} -0.4330621949635189
4.574919438578312 7.297804356324441


In [47]:
params={
    'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
#     'cat_smooth': [1, 10, 15, 20, 35]
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50, learning_rate=0.1, lambda_l1=0.3,lambda_l2=8)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
S:\anaconda\envs\tf\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done  75 out of  75 | elapsed:  2.0min finished


{'bagging_fraction': 0.6, 'feature_fraction': 0.6} -0.43298128261992097
4.764058910910964 7.474783919887232


In [49]:
params={
    'cat_smooth': [0.1, 0.3, 0.7, 1, 3, 5, 8]
}
gbm = LGBMClassifier(n_jobs=8, max_depth=20, num_leaves=50, learning_rate=0.1, lambda_l1=0.3,lambda_l2=8)

gsearch1 = GridSearchCV(gbm, param_grid=params, scoring='neg_log_loss', cv=3, verbose=1, n_jobs=3)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_, gsearch1.best_score_)
t_model(gsearch1.best_estimator_)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  21 out of  21 | elapsed:   48.9s finished


{'cat_smooth': 0.1} -0.4330621949635189
4.574919438578312 7.297804356324441
